In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
master_file = "Master.csv"
master_df = pd.read_csv(master_file)
master_df.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,220.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [3]:
salaries_file = "Salaries.csv"
salaries_df = pd.read_csv(salaries_file)
salaries_df.head()

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000


In [4]:
batting_file = "Batting.csv"
batting_df = pd.read_csv(batting_file)
batting_df.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NaN,25,118.0,30.0,32.0,6.0,...,13.0,8.0,1.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN
2,allisar01,1871,1,CL1,NaN,29,137.0,28.0,40.0,4.0,...,19.0,3.0,1.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN
3,allisdo01,1871,1,WS3,NaN,27,133.0,28.0,44.0,10.0,...,27.0,1.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN
4,ansonca01,1871,1,RC1,NaN,25,120.0,29.0,39.0,11.0,...,16.0,6.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN


In [5]:
# Create a filtered dataframe from specific columns
master_cols = ["playerID", "nameLast", "nameGiven", "birthYear", "debut", "finalGame"]
master_transformed = master_df[master_cols].copy()

# Rename the column headers
master_transformed = master_transformed.rename(columns={"nameLast": "Last_Name",
                                                        "nameGiven": "First_Name",
                                                        "birthYear": "Birth_Year",
                                                        "debut": "Debut_Game",
                                                        "finalGame": "Final_Game"})

# Clean the data by dropping duplicates and setting the index
master_transformed.drop_duplicates("playerID", inplace=True)
master_transformed.set_index("playerID", inplace=True)

master_transformed.head()

,Last_Name,First_Name,Birth_Year,Debut_Game,Final_Game
playerID,,,,,
aardsda01,Aardsma,David Allan,1981.0,2004-04-06,2015-08-23
aaronha01,Aaron,Henry Louis,1934.0,1954-04-13,1976-10-03
aaronto01,Aaron,Tommie Lee,1939.0,1962-04-10,1971-09-26
aasedo01,Aase,Donald William,1954.0,1977-07-26,1990-10-03
abadan01,Abad,Fausto Andres,1972.0,2001-09-10,2006-04-13


In [6]:
# Run basic calculations
player_salary_total = salaries_df.groupby(["playerID"]).sum()["salary"].rename("Total_Earnings")
player_seasons_played = salaries_df.groupby(["playerID"]).count()["salary"].rename("Seasons_Played")

# Convert to DataFrame
salaries_transformed = pd.DataFrame({"Seasons_Played": player_seasons_played,
                                     "Total_Earnings": player_salary_total})

salaries_transformed.head()

,Seasons_Played,Total_Earnings
playerID,,
aardsda01,7,9259750
aasedo01,4,2300000
abadan01,1,327000
abadfe01,4,2516400
abbotje01,4,985000


In [7]:
# Clean the data by dropping old rows
batting_df.drop(batting_df[batting_df.yearID < 1985].index, inplace=True)

# Run basic calculations
player_games_played = batting_df.groupby(["playerID"]).sum()["G"].rename("Games_Played")

# Convert to DataFrame
batting_transformed = pd.DataFrame({"Games_Played": player_games_played})

batting_transformed.head()

,Games_Played
playerID,
aardsda01,331
aasedo01,243
abadan01,15
abadfe01,258
abbotje01,233


In [8]:
# Create database connection
connection_string = "root:mysqlroot@localhost/etl_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [9]:
# Confirm tables
engine.table_names()

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# Load master DataFrame into database
master_transformed.to_sql(name='master', con=engine, if_exists='append', index=True)

In [ ]:
# Load salaries DataFrame into database
salaries_transformed.to_sql(name='salaries', con=engine, if_exists='append', index=True)

In [ ]:
# Load batting DataFrame into database
batting_transformed.to_sql(name='batting', con=engine, if_exists='append', index=True)